In [1]:
from IPython.display import clear_output
import pandas as pd 
import subprocess #run R code from subprocess
import numpy as np
from os import listdir # get dir list
from os.path import isfile, join # get dir list


import concurrent.futures
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, cross_validate
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

In [2]:
def getFolderFilesList(path):
    return [filename for filename in listdir(path) if isfile(join(path, filename))]

# def runKNN(dataset):
#     scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro', 'roc_auc']
#     model = KNeighborsClassifier()
#     df = dataset
#     X, y = df.drop('class', axis=1), df['class']
#     scores = cross_validate(model, X, y, cv=5,scoring=scoring, n_jobs=15)
#     return scores

# def runDecisionTree(dataset):
#     scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro', 'roc_auc']
#     model = DecisionTreeClassifier()
#     df = dataset
#     X, y = df.drop('class', axis=1), df['class']
#     scores = cross_validate(model, X, y, cv=5,scoring=scoring, n_jobs=15)
#     return scores

# def runRF(dataset):
#     scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro', 'roc_auc']
#     model = RandomForestClassifier()
#     df = dataset
#     X, y = df.drop('class', axis=1), df['class']
#     scores = cross_validate(model, X, y, cv=5,scoring=scoring, n_jobs=15)
#     return scores 

In [3]:

datasetsFilesNames = getFolderFilesList('NoisyDatasetsAfterFilters/GE-3')
runKNN(pd.read_csv(f'NoisyDatasetsAfterFilters/GE-3/{datasetsFilesNames[0]}'))
runDecisionTree(pd.read_csv(f'NoisyDatasetsAfterFilters/GE-3/{datasetsFilesNames[0]}'))
runRF(pd.read_csv(f'NoisyDatasetsAfterFilters/GE-3/{datasetsFilesNames[0]}'))

NameError: name 'runKNN' is not defined

In [3]:
def DictTosplitedDF(scoreDict):
    a = pd.DataFrame.from_dict(scoreDict, orient='index')
    cols = a.columns
    for i in cols:
        df_aux = pd.DataFrame((a[i].tolist()), columns= [f'{i}_{n}' for n in range(5)], index=a.index)
        df_aux = df_aux.fillna(0)
        df_aux[f'{i}_mean'] = np.mean(df_aux, axis=1)
        df_aux[f'{i}_std'] = np.std(df_aux, axis=1)
        a = a.drop(i, axis=1)
        a = a.join(df_aux)
    return a
    
def runClassifiers(file, folder):
    def runKNN(dataset):
        scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro', 'roc_auc']
        model = KNeighborsClassifier()
        df = dataset
        X, y = df.drop('class', axis=1), df['class']
        scores = cross_validate(model, X, y, cv=5,scoring=scoring, n_jobs=15)
        return scores

    def runDecisionTree(dataset):
        scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro', 'roc_auc']
        model = DecisionTreeClassifier()
        df = dataset
        X, y = df.drop('class', axis=1), df['class']
        scores = cross_validate(model, X, y, cv=5,scoring=scoring, n_jobs=15)
        return scores

    def runRF(dataset):
        scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro', 'roc_auc']
        model = RandomForestClassifier()
        df = dataset
        X, y = df.drop('class', axis=1), df['class']
        scores = cross_validate(model, X, y, cv=5,scoring=scoring, n_jobs=15)
        return scores     
    df = pd.read_csv(f'{folder}/{file}')
    knnScore ={}
    DTScore = {}
    RFScore = {}
    try:
        knnScore[file] = runKNN(df)
    except:
        nanArray = [np.nan for _ in range(5)]
        knnScore[file] = {'fit_time':nanArray, 'score_time': nanArray, 'test_accuracy': nanArray,
                    'test_precision_macro': nanArray,'test_recall_macro': nanArray,
                    'test_f1_macro': nanArray, 'test_roc_auc': nanArray}
    try:
        DTScore[file] = runDecisionTree(df)
    except:
        nanArray = [np.nan for _ in range(5)]
        DTScore[file] = {'fit_time':nanArray, 'score_time': nanArray, 'test_accuracy': nanArray,
                    'test_precision_macro': nanArray,'test_recall_macro': nanArray,
                    'test_f1_macro': nanArray, 'test_roc_auc': nanArray}
    try:
        RFScore[file] = runRF(df)
    except:
        nanArray = [np.nan for _ in range(5)]
        RFScore[file] = {'fit_time':nanArray, 'score_time': nanArray, 'test_accuracy': nanArray,
                    'test_precision_macro': nanArray,'test_recall_macro': nanArray,
                    'test_f1_macro': nanArray, 'test_roc_auc': nanArray}

    return knnScore, DTScore, RFScore, file


def runClassifiersSVM(file, folder):
    def runSVM(dataset):
        scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro', 'roc_auc']
        model = svm.SVC()
        df = dataset
        X, y = df.drop('class', axis=1), df['class']
        scores = cross_validate(model, X, y, cv=5,scoring=scoring, n_jobs=15)
        return scores

    # def runDecisionTree(dataset):
    #     scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro', 'roc_auc']
    #     model = DecisionTreeClassifier()
    #     df = dataset
    #     X, y = df.drop('class', axis=1), df['class']
    #     scores = cross_validate(model, X, y, cv=5,scoring=scoring, n_jobs=15)
    #     return scores

    # def runRF(dataset):
    #     scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro', 'roc_auc']
    #     model = RandomForestClassifier()
    #     df = dataset
    #     X, y = df.drop('class', axis=1), df['class']
    #     scores = cross_validate(model, X, y, cv=5,scoring=scoring, n_jobs=15)
    #     return scores     
    df = pd.read_csv(f'{folder}/{file}')
    # knnScore ={}
    # DTScore = {}
    SVMScore = {}
    try:
        SVMScore[file] = runSVM(df)
    except:
        nanArray = [np.nan for _ in range(5)]
        SVMScore[file] = {'fit_time':nanArray, 'score_time': nanArray, 'test_accuracy': nanArray,
                    'test_precision_macro': nanArray,'test_recall_macro': nanArray,
                    'test_f1_macro': nanArray, 'test_roc_auc': nanArray}
    # try:
    #     DTScore[file] = runDecisionTree(df)
    # except:
    #     nanArray = [np.nan for _ in range(5)]
    #     DTScore[file] = {'fit_time':nanArray, 'score_time': nanArray, 'test_accuracy': nanArray,
    #                 'test_precision_macro': nanArray,'test_recall_macro': nanArray,
    #                 'test_f1_macro': nanArray, 'test_roc_auc': nanArray}
    # try:
    #     RFScore[file] = runRF(df)
    # except:
    #     nanArray = [np.nan for _ in range(5)]
    #     RFScore[file] = {'fit_time':nanArray, 'score_time': nanArray, 'test_accuracy': nanArray,
    #                 'test_precision_macro': nanArray,'test_recall_macro': nanArray,
    #                 'test_f1_macro': nanArray, 'test_roc_auc': nanArray}

    return SVMScore, file

In [4]:
# folderList = ['NoisyDatasetsAfterFilters/GE-3', 'NoisyDatasetsAfterFilters/GE-5', 'NoisyDatasetsAfterFilters/GE-7', 'NoisyDatasetsAfterFilters/GE-9', 'NoisyDatasetsAfterFilters/GE-11',
# 'NoisyDatasetsAfterFilters/HARF-70', 'NoisyDatasetsAfterFilters/HARF-75', 'NoisyDatasetsAfterFilters/HARF-80', 'NoisyDatasetsAfterFilters/HARF-85', 'NoisyDatasetsAfterFilters/HARF-90', 
# 'NoisyDatasetsAfterFilters/ORBoost-3', 'NoisyDatasetsAfterFilters/ORBoost-7', 'NoisyDatasetsAfterFilters/ORBoost-11', 'NoisyDatasetsAfterFilters/ORBoost-15', 'NoisyDatasetsAfterFilters/ORBoost-19',
# ]
# folderList = ['NoisyDatasetsAfterFilters/dynamicCF', 'NoisyDatasetsAfterFilters/AENN', 'NoisyDatasetsAfterFilters/edgeBoostFilter-5',
# 'NoisyDatasetsAfterFilters/edgeBoostFilter-10', 'NoisyDatasetsAfterFilters/edgeBoostFilter-20', 'NoisyDatasetsAfterFilters/hybridRepairFilter',
# 'NoisyDatasetsAfterFilters/PRISM']

# folderList = ['NoisyDatasetsAfterFilters/dynamicCF', 'NoisyDatasetsAfterFilters/AENN-3', 'NoisyDatasetsAfterFilters/AENN-5', 'NoisyDatasetsAfterFilters/AENN-7',
#  'NoisyDatasetsAfterFilters/AENN-9', 'NoisyDatasetsAfterFilters/AENN-11', 'NoisyDatasetsAfterFilters/hybridRepairFilter',
# 'NoisyDatasetsAfterFilters/PRISM']

# folderList = ['NoisyDatasetsAfterFilters/HARF-70', 'NoisyDatasetsAfterFilters/HARF-75', 'NoisyDatasetsAfterFilters/HARF-80',
#          'NoisyDatasetsAfterFilters/HARF-85', 'NoisyDatasetsAfterFilters/HARF-90', 'NoisyDatasetsAfterFilters/edgeBoostFilter-15']
folderList = ['NoisyDatasets']

start = time.perf_counter()
for folderIndex, folder in enumerate(folderList):
    datasetsFilesNames = getFolderFilesList(folder)
    smallSet = datasetsFilesNames
    knnScores ={}
    DTScores={}
    RFScores={}
    quantityOfFiles = len(datasetsFilesNames)
    count = 0
    startFolder = time.perf_counter()
    with concurrent.futures.ProcessPoolExecutor(10) as executor:
    #                 results = executor.map(runMFE, smallSet)
        results = [executor.submit(runClassifiers, file, folder) for file in smallSet]
        for s in concurrent.futures.as_completed(results):
            clear_output(wait=True)
            knnScore, DTScore, RFScore, f = s.result()
            knnScores[f] = knnScore[f]
            DTScores[f] = DTScore[f]
            RFScores[f] = RFScore[f]

            count+=1
            end = time.perf_counter()
            print(f'executando {count} / {quantityOfFiles} pasta {folderIndex} - {folder}')
            print(f'tempo de execução: {int((end-startFolder)/60)} min {int((end-startFolder)%60)} s')
            print(f'tempo total: {int((end-start)/60)} min {int((end-start)%60)} s' )

    # runClassifiers(datasetsFilesNames[0], folder)
    DictTosplitedDF(knnScores).to_csv(f'Results/{folder.replace("/", "_")}_resultsKNN.csv.zip')
    DictTosplitedDF(DTScores).to_csv(f'Results/{folder.replace("/", "_")}_resultsDT.csv.zip')
    DictTosplitedDF(RFScores).to_csv(f'Results/{folder.replace("/", "_")}_resultsRF.csv.zip')



executando 10740 / 10740 pasta 0 - NoisyDatasets
tempo de execução: 12 min 15 s
tempo total: 12 min 15 s


/home/lpfgarcia/pedropio/NoiseSelection/NoiseEnvPy/lib/python3.8/site-packages/joblib/externals/loky/backend/resource_tracker.py:318: UserWarning: resource_tracker: There appear to be 3132 leaked folder objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/home/lpfgarcia/pedropio/NoiseSelection/NoiseEnvPy/lib/python3.8/site-packages/joblib/externals/loky/backend/resource_tracker.py:318: UserWarning: resource_tracker: There appear to be 3501 leaked folder objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/home/lpfgarcia/pedropio/NoiseSelection/NoiseEnvPy/lib/python3.8/site-packages/joblib/externals/loky/backend/resource_tracker.py:318: UserWarning: resource_tracker: There appear to be 3321 leaked folder objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/home/lpfgarcia/pedropio/NoiseSelection/NoiseEnvPy/lib/python3.8/site-packages/joblib/externals/loky/backend/resource_

In [23]:
def readfile(file, folder):
    df = pd.read_csv(f'{folder}/{file}')
    return df, file
folderList = ['NoisyDatasetsAfterFilters/dynamicCF', 'NoisyDatasetsAfterFilters/AENN', 'NoisyDatasetsAfterFilters/edgeBoostFilter-5',
'NoisyDatasetsAfterFilters/edgeBoostFilter-10', 'NoisyDatasetsAfterFilters/edgeBoostFilter-20', 'NoisyDatasetsAfterFilters/hybridRepairFilter',
'NoisyDatasetsAfterFilters/PRISM']
start = time.perf_counter()
for folderIndex, folder in enumerate(folderList):
    datasetsFilesNames = getFolderFilesList(folder)
    smallSet = datasetsFilesNames[0:200]
    knnScores ={}
    DTScores={}
    RFScores={}
    quantityOfFiles = len(datasetsFilesNames)
    count = 0
    startFolder = time.perf_counter()
    with concurrent.futures.ProcessPoolExecutor(10) as executor:
    #                 results = executor.map(runMFE, smallSet)
        results = [executor.submit(readfile, file, folder) for file in smallSet]
        for s in concurrent.futures.as_completed(results):
            clear_output(wait=True)
            # df = pd.read_csv(f'{folder}/{file}')
            df, file = s.result()
            try:
                knnScores[file] = runKNN(df)
            except:
                nanArray = [np.nan for _ in range(5)]
                knnScores[file] = {'fit_time':nanArray, 'score_time': nanArray, 'test_accuracy': nanArray,
                            'test_precision_macro': nanArray,'test_recall_macro': nanArray,
                            'test_f1_macro': nanArray, 'test_roc_auc': nanArray}
            try:
                DTScores[file] = runDecisionTree(df)
            except:
                nanArray = [np.nan for _ in range(5)]
                DTScores[file] = {'fit_time':nanArray, 'score_time': nanArray, 'test_accuracy': nanArray,
                            'test_precision_macro': nanArray,'test_recall_macro': nanArray,
                            'test_f1_macro': nanArray, 'test_roc_auc': nanArray}
            try:
                RFScores[file] = runRF(df)
            except:
                nanArray = [np.nan for _ in range(5)]
                RFScores[file] = {'fit_time':nanArray, 'score_time': nanArray, 'test_accuracy': nanArray,
                            'test_precision_macro': nanArray,'test_recall_macro': nanArray,
                            'test_f1_macro': nanArray, 'test_roc_auc': nanArray}
            
            count+=1    
            end = time.perf_counter()
            print(f'executando {count} / {quantityOfFiles} pasta {folderIndex} - {folder}')
            print(f'tempo de execução: {int((end-startFolder)/60)} min {int((end-startFolder)%60)} s')
            print(f'tempo total: {int((end-start)/60)} min {int((end-start)%60)} s' )

executando 200 / 10650 pasta 0 - NoisyDatasetsAfterFilters/dynamicCF
tempo de execução: 1 min 25 s
tempo total: 1 min 25 s


In [24]:
folderList = ['NoisyDatasetsAfterFilters/dynamicCF']
# , 'NoisyDatasetsAfterFilters/AENN', 'NoisyDatasetsAfterFilters/edgeBoostFilter-5',
# 'NoisyDatasetsAfterFilters/edgeBoostFilter-10', 'NoisyDatasetsAfterFilters/edgeBoostFilter-20', 'NoisyDatasetsAfterFilters/hybridRepairFilter',
# 'NoisyDatasetsAfterFilters/PRISM']
start = time.perf_counter()
for folderIndex, folder in enumerate(folderList):
    datasetsFilesNames = getFolderFilesList(folder)
    smallSet = datasetsFilesNames[0:200]
    knnScores ={}
    DTScores={}
    RFScores={}
    quantityOfFiles = len(datasetsFilesNames)
    count = 0
    startFolder = time.perf_counter()
    for file in smallSet:
        clear_output(wait=True)
        df = pd.read_csv(f'{folder}/{file}')
        try:
            knnScores[file] = runKNN(df)
        except:
            nanArray = [np.nan for _ in range(5)]
            knnScores[file] = {'fit_time':nanArray, 'score_time': nanArray, 'test_accuracy': nanArray,
                        'test_precision_macro': nanArray,'test_recall_macro': nanArray,
                        'test_f1_macro': nanArray, 'test_roc_auc': nanArray}
        try:
            DTScores[file] = runDecisionTree(df)
        except:
            nanArray = [np.nan for _ in range(5)]
            DTScores[file] = {'fit_time':nanArray, 'score_time': nanArray, 'test_accuracy': nanArray,
                        'test_precision_macro': nanArray,'test_recall_macro': nanArray,
                        'test_f1_macro': nanArray, 'test_roc_auc': nanArray}
        try:
            RFScores[file] = runRF(df)
        except:
            nanArray = [np.nan for _ in range(5)]
            RFScores[file] = {'fit_time':nanArray, 'score_time': nanArray, 'test_accuracy': nanArray,
                        'test_precision_macro': nanArray,'test_recall_macro': nanArray,
                        'test_f1_macro': nanArray, 'test_roc_auc': nanArray}
        
        count+=1    
        end = time.perf_counter()
        print(f'executando {count} / {quantityOfFiles} pasta {folderIndex} - {folder}')
        print(f'tempo de execução: {int((end-startFolder)/60)} min {int((end-startFolder)%60)} s')
        print(f'tempo total: {int((end-start)/60)} min {int((end-start)%60)} s' )

executando 200 / 10650 pasta 0 - NoisyDatasetsAfterFilters/dynamicCF
tempo de execução: 1 min 24 s
tempo total: 1 min 24 s


In [27]:
display(DictTosplitedDF(knnScores))
display(DictTosplitedDF(RFScores))
display(DictTosplitedDF(DTScores))

,fit_time_0,fit_time_1,fit_time_2,fit_time_3,fit_time_4,fit_time_mean,fit_time_std,score_time_0,score_time_1,score_time_2,...,test_f1_macro_4,test_f1_macro_mean,test_f1_macro_std,test_roc_auc_0,test_roc_auc_1,test_roc_auc_2,test_roc_auc_3,test_roc_auc_4,test_roc_auc_mean,test_roc_auc_std
728_analcatdata_supreme_10_2.csv.gz,0.002006,0.001950,0.002450,0.003258,0.002687,0.002470,0.000438,0.005601,0.005812,0.008042,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
885_transplant_5_1.csv.gz,0.003315,0.002412,0.012921,0.002506,0.002195,0.004670,0.003782,0.010034,0.006973,0.023019,...,0.823077,0.944447,0.065879,0.937500,1.000000,1.000000,1.000000,0.933333,0.974167,0.028908
1012_flags_20_6.csv.gz,0.002828,0.003941,0.003900,0.004458,0.003777,0.003781,0.000484,0.010570,0.013283,0.025788,...,0.470057,0.501497,0.043400,0.633152,0.459239,0.517663,0.479620,0.515942,0.521123,0.054987
860_vinnie_20_0.csv.gz,0.002436,0.002258,0.002292,0.003417,0.003125,0.002705,0.000433,0.008044,0.007554,0.007852,...,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
1498_sa-heart_5_0.csv.gz,0.003525,0.002596,0.003566,0.003513,0.003294,0.003299,0.000332,0.011550,0.008115,0.011585,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855_fri_c4_500_10_20_3.csv.gz,0.003999,0.003765,0.002190,0.005039,0.002009,0.003400,0.001047,0.014162,0.009397,0.008525,...,0.983154,0.930461,0.028961,0.983440,0.957265,0.971688,0.971154,0.998352,0.976380,0.012565
43946_eye_movements_20_6.csv.gz,0.005939,0.006821,0.002810,0.004113,0.005180,0.004973,0.001279,0.090422,0.087577,0.111460,...,0.238382,0.315151,0.091023,0.515818,0.207315,0.248834,0.277903,0.200035,0.289981,0.106274
977_letter_20_6.csv.gz,0.004369,0.003266,0.003499,0.005182,0.006573,0.004578,0.001101,2.024473,1.839097,1.998322,...,1.000000,0.993095,0.005932,1.000000,0.983333,0.983333,1.000000,1.000000,0.993333,0.007454
833_bank32nh_5_0.csv.gz,0.007222,0.003561,0.006129,0.004550,0.004511,0.005194,0.001193,0.057398,0.060457,0.064806,...,0.952284,0.956798,0.002963,0.993667,0.995474,0.990353,0.994062,0.992419,0.993195,0.001573


,fit_time_0,fit_time_1,fit_time_2,fit_time_3,fit_time_4,fit_time_mean,fit_time_std,score_time_0,score_time_1,score_time_2,...,test_f1_macro_4,test_f1_macro_mean,test_f1_macro_std,test_roc_auc_0,test_roc_auc_1,test_roc_auc_2,test_roc_auc_3,test_roc_auc_4,test_roc_auc_mean,test_roc_auc_std
728_analcatdata_supreme_10_2.csv.gz,0.164014,0.138189,0.141972,0.086205,0.088026,0.123681,0.028424,0.030838,0.027277,0.026739,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
885_transplant_5_1.csv.gz,0.185200,0.088748,0.170631,0.110729,0.166172,0.144296,0.034297,0.036812,0.018420,0.033178,...,0.865497,0.963207,0.047904,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
1012_flags_20_6.csv.gz,0.102091,0.101560,0.154051,0.102814,0.124304,0.116964,0.018655,0.019403,0.019904,0.028452,...,0.532308,0.526789,0.032902,0.501359,0.633152,0.648098,0.633152,0.689855,0.621123,0.057857
860_vinnie_20_0.csv.gz,0.092993,0.109099,0.166477,0.164150,0.111006,0.128745,0.027857,0.018249,0.021898,0.036250,...,0.325000,0.865000,0.246475,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
1498_sa-heart_5_0.csv.gz,0.164510,0.144981,0.139125,0.165768,0.111841,0.145245,0.018008,0.040455,0.025346,0.023740,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855_fri_c4_500_10_20_3.csv.gz,0.109644,0.114854,0.173809,0.176253,0.109879,0.136888,0.028490,0.018129,0.018379,0.028449,...,0.882077,0.926852,0.032852,0.994658,0.985043,0.988782,0.975962,0.976374,0.984164,0.006585
43946_eye_movements_20_6.csv.gz,1.426572,1.469765,1.408618,1.433589,1.490077,1.445724,0.027191,0.070410,0.062528,0.066240,...,0.134680,0.250315,0.111342,0.481795,0.146620,0.145181,0.146921,0.098758,0.203855,0.127969
977_letter_20_6.csv.gz,0.384899,0.385883,0.373170,0.605966,0.597840,0.469552,0.098761,0.051967,0.051455,0.051049,...,1.000000,0.994806,0.003873,1.000000,0.999978,0.983183,1.000000,1.000000,0.996632,0.006139
833_bank32nh_5_0.csv.gz,1.933305,2.005114,2.118229,2.127234,2.163118,2.069400,0.078705,0.048676,0.046299,0.044627,...,0.957979,0.954186,0.003879,0.994731,0.996880,0.994871,0.995703,0.995506,0.995538,0.000698


,fit_time_0,fit_time_1,fit_time_2,fit_time_3,fit_time_4,fit_time_mean,fit_time_std,score_time_0,score_time_1,score_time_2,...,test_f1_macro_4,test_f1_macro_mean,test_f1_macro_std,test_roc_auc_0,test_roc_auc_1,test_roc_auc_2,test_roc_auc_3,test_roc_auc_4,test_roc_auc_mean,test_roc_auc_std
728_analcatdata_supreme_10_2.csv.gz,0.002475,0.002855,0.003579,0.003330,0.003225,0.003093,0.000353,0.005277,0.007058,0.008351,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
885_transplant_5_1.csv.gz,0.002280,0.002694,0.002611,0.003004,0.002305,0.002579,0.000245,0.004916,0.005690,0.005699,...,0.780952,0.936022,0.079252,0.875000,1.000000,1.000000,1.000000,0.833333,0.941667,0.066319
1012_flags_20_6.csv.gz,0.004049,0.004085,0.005855,0.004167,0.003753,0.004382,0.000684,0.006710,0.006897,0.010066,...,0.581805,0.564885,0.035621,0.523098,0.631793,0.535326,0.557065,0.581159,0.565688,0.035165
860_vinnie_20_0.csv.gz,0.003081,0.002361,0.002235,0.002155,0.003049,0.002576,0.000369,0.007525,0.005277,0.005057,...,0.325000,0.865000,0.246475,1.000000,1.000000,1.000000,1.000000,0.500000,0.900000,0.182574
1498_sa-heart_5_0.csv.gz,0.003616,0.002533,0.002743,0.003800,0.002824,0.003103,0.000462,0.008518,0.005228,0.006044,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855_fri_c4_500_10_20_3.csv.gz,0.004768,0.003163,0.003177,0.007330,0.003584,0.004404,0.001438,0.005278,0.006543,0.007761,...,0.898333,0.878495,0.012216,0.870726,0.851496,0.862179,0.897436,0.894505,0.875269,0.016424
43946_eye_movements_20_6.csv.gz,0.121073,0.185825,0.197613,0.159095,0.168641,0.166450,0.024029,0.008073,0.012039,0.011499,...,0.328097,0.380061,0.046867,0.481408,0.391424,0.373121,0.387434,0.344094,0.395496,0.042042
977_letter_20_6.csv.gz,0.031298,0.019465,0.026979,0.023531,0.026674,0.025589,0.003593,0.009633,0.006674,0.006085,...,1.000000,0.987960,0.007999,0.999833,0.966667,0.966499,0.983333,1.000000,0.983266,0.013608
833_bank32nh_5_0.csv.gz,0.147671,0.172812,0.156279,0.167077,0.182308,0.165230,0.011113,0.007335,0.010037,0.011311,...,0.948090,0.949194,0.007898,0.943269,0.960154,0.932345,0.950478,0.948673,0.946984,0.008333


### SVM

In [4]:
# folderList = ['NoisyDatasetsAfterFilters/GE-3', 'NoisyDatasetsAfterFilters/GE-5', 'NoisyDatasetsAfterFilters/GE-7', 'NoisyDatasetsAfterFilters/GE-9', 'NoisyDatasetsAfterFilters/GE-11',
# 'NoisyDatasetsAfterFilters/HARF-70', 'NoisyDatasetsAfterFilters/HARF-75', 'NoisyDatasetsAfterFilters/HARF-80', 'NoisyDatasetsAfterFilters/HARF-85', 'NoisyDatasetsAfterFilters/HARF-90', 
# 'NoisyDatasetsAfterFilters/ORBoost-3', 'NoisyDatasetsAfterFilters/ORBoost-7', 'NoisyDatasetsAfterFilters/ORBoost-11', 'NoisyDatasetsAfterFilters/ORBoost-15', 'NoisyDatasetsAfterFilters/ORBoost-19',
# ]
# folderList = ['NoisyDatasetsAfterFilters/dynamicCF', 'NoisyDatasetsAfterFilters/AENN', 'NoisyDatasetsAfterFilters/edgeBoostFilter-5',
# 'NoisyDatasetsAfterFilters/edgeBoostFilter-10', 'NoisyDatasetsAfterFilters/edgeBoostFilter-20', 'NoisyDatasetsAfterFilters/hybridRepairFilter',
# 'NoisyDatasetsAfterFilters/PRISM']

# folderList = ['NoisyDatasetsAfterFilters/dynamicCF', 'NoisyDatasetsAfterFilters/AENN-3', 'NoisyDatasetsAfterFilters/AENN-5', 'NoisyDatasetsAfterFilters/AENN-7',
#  'NoisyDatasetsAfterFilters/AENN-9', 'NoisyDatasetsAfterFilters/AENN-11', 'NoisyDatasetsAfterFilters/hybridRepairFilter',
# 'NoisyDatasetsAfterFilters/PRISM']

# folderList = ['NoisyDatasetsAfterFilters/HARF-70', 'NoisyDatasetsAfterFilters/HARF-75', 'NoisyDatasetsAfterFilters/HARF-80',
#          'NoisyDatasetsAfterFilters/HARF-85', 'NoisyDatasetsAfterFilters/HARF-90', 'NoisyDatasetsAfterFilters/edgeBoostFilter-15']
folderList = ['NoisyDatasets']

# folderList = ['NoisyDatasetsAfterFilters/GE-3', 'NoisyDatasetsAfterFilters/GE-5', 'NoisyDatasetsAfterFilters/GE-7', 'NoisyDatasetsAfterFilters/GE-9', 'NoisyDatasetsAfterFilters/GE-11',
# 'NoisyDatasetsAfterFilters/HARF-70', 'NoisyDatasetsAfterFilters/HARF-75', 'NoisyDatasetsAfterFilters/HARF-80', 'NoisyDatasetsAfterFilters/HARF-85', 'NoisyDatasetsAfterFilters/HARF-90', 
# 'NoisyDatasetsAfterFilters/ORBoost-3', 'NoisyDatasetsAfterFilters/ORBoost-7', 'NoisyDatasetsAfterFilters/ORBoost-11', 'NoisyDatasetsAfterFilters/ORBoost-15', 'NoisyDatasetsAfterFilters/ORBoost-19',
# 'NoisyDatasetsAfterFilters/dynamicCF', 'NoisyDatasetsAfterFilters/edgeBoostFilter-5', 'NoisyDatasetsAfterFilters/edgeBoostFilter-10', 
# 'NoisyDatasetsAfterFilters/edgeBoostFilter-15' ,'NoisyDatasetsAfterFilters/edgeBoostFilter-20', 'NoisyDatasetsAfterFilters/hybridRepairFilter',
# 'NoisyDatasetsAfterFilters/PRISM', 'NoisyDatasetsAfterFilters/AENN-3', 'NoisyDatasetsAfterFilters/AENN-5', 'NoisyDatasetsAfterFilters/AENN-7',
#  'NoisyDatasetsAfterFilters/AENN-9', 'NoisyDatasetsAfterFilters/AENN-11',
# ]

start = time.perf_counter()
for folderIndex, folder in enumerate(folderList):
    datasetsFilesNames = getFolderFilesList(folder)
    smallSet = datasetsFilesNames
    knnScores ={}
    DTScores={}
    SVMScores={}
    quantityOfFiles = len(datasetsFilesNames)
    count = 0
    startFolder = time.perf_counter()
    with concurrent.futures.ProcessPoolExecutor(20) as executor:
    #                 results = executor.map(runMFE, smallSet)
        results = [executor.submit(runClassifiersSVM, file, folder) for file in smallSet]
        for s in concurrent.futures.as_completed(results):
            clear_output(wait=True)
            svmScore, f = s.result()
            # knnScores[f] = knnScore[f]
            # DTScores[f] = DTScore[f]
            SVMScores[f] = svmScore[f]

            count+=1
            end = time.perf_counter()
            print(f'executando {count} / {quantityOfFiles} pasta {folderIndex} - {folder}')
            print(f'tempo de execução: {int((end-startFolder)/60)} min {int((end-startFolder)%60)} s')
            print(f'tempo total: {int((end-start)/60)} min {int((end-start)%60)} s' )

    # runClassifiers(datasetsFilesNames[0], folder)
    DictTosplitedDF(SVMScores).to_csv(f'Results/{folder.replace("/", "_")}_resultsSVM.csv.zip')
    # DictTosplitedDF(DTScores).to_csv(f'Results/{folder.replace("/", "_")}_resultsDT.csv.zip')
    # DictTosplitedDF(RFScores).to_csv(f'Results/{folder.replace("/", "_")}_resultsRF.csv.zip')


executando 10740 / 10740 pasta 0 - NoisyDatasets
tempo de execução: 12 min 21 s
tempo total: 12 min 21 s


/home/lpfgarcia/pedropio/NoiseSelection/NoiseEnvPy/lib/python3.8/site-packages/joblib/externals/loky/backend/resource_tracker.py:318: UserWarning: resource_tracker: There appear to be 467 leaked folder objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/home/lpfgarcia/pedropio/NoiseSelection/NoiseEnvPy/lib/python3.8/site-packages/joblib/externals/loky/backend/resource_tracker.py:318: UserWarning: resource_tracker: There appear to be 634 leaked folder objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/home/lpfgarcia/pedropio/NoiseSelection/NoiseEnvPy/lib/python3.8/site-packages/joblib/externals/loky/backend/resource_tracker.py:318: UserWarning: resource_tracker: There appear to be 459 leaked folder objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/home/lpfgarcia/pedropio/NoiseSelection/NoiseEnvPy/lib/python3.8/site-packages/joblib/externals/loky/backend/resource_tra

In [6]:
pd.read_csv('Results/NoisyDatasetsAfterFilters_AENN-3_resultsSVM.csv.zip')

,Unnamed: 0,fit_time_0,fit_time_1,fit_time_2,fit_time_3,fit_time_4,fit_time_mean,fit_time_std,score_time_0,score_time_1,...,test_f1_macro_4,test_f1_macro_mean,test_f1_macro_std,test_roc_auc_0,test_roc_auc_1,test_roc_auc_2,test_roc_auc_3,test_roc_auc_4,test_roc_auc_mean,test_roc_auc_std
0,1012_flags_20_6.csv.gz,0.006560,0.007560,0.007122,0.006438,0.007881,0.007112,0.000509,0.011017,0.014572,...,0.410256,0.413759,0.001599,0.252101,0.571429,0.504202,0.394958,0.258929,0.396324,0.116889
1,1498_sa-heart_5_0.csv.gz,0.006973,0.008323,0.009060,0.005992,0.008289,0.007728,0.001002,0.011530,0.012293,...,0.434343,0.433709,0.002404,0.882867,0.842657,0.788909,0.778175,0.837209,0.825964,0.034881
2,860_vinnie_20_0.csv.gz,0.006407,0.007914,0.005333,0.005284,0.007132,0.006414,0.000932,0.011883,0.015322,...,0.387097,0.437410,0.100296,1.000000,0.969444,0.997222,0.797619,1.000000,0.952857,0.071632
3,1067_kc1_5_1.csv.gz,0.034086,0.033268,0.033112,0.041545,0.033743,0.035151,0.002936,0.023371,0.023493,...,0.536478,0.566503,0.028627,0.765621,0.570196,0.384706,0.689542,0.581176,0.598248,0.117634
4,942_chscase_health_5_3.csv.gz,0.004357,0.008258,0.006126,0.006399,0.004313,0.005891,0.001339,0.008403,0.013521,...,0.708333,0.517619,0.168501,0.666667,0.500000,0.666667,0.750000,1.000000,0.716667,0.149071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10735,923_visualizing_soil_10_7.csv.gz,1.332371,1.756384,1.938203,1.797907,1.901517,1.745276,0.197912,0.908007,0.509106,...,0.863849,0.783941,0.180538,0.615068,0.932647,0.947243,0.979801,0.988711,0.892694,0.128098
10736,977_letter_20_6.csv.gz,3.071729,2.872044,3.330509,3.217116,2.859006,3.070081,0.169910,0.616551,0.645294,...,0.680856,0.683051,0.013805,0.695542,0.697144,0.746822,0.705961,0.706357,0.710365,0.017123
10737,846_elevators_5_8.csv.gz,4.424062,4.402395,4.494762,4.002537,4.656515,4.396054,0.197173,2.433346,2.764378,...,0.448638,0.436313,0.007080,0.782533,0.808492,0.878138,0.756678,0.754588,0.796086,0.041523
10738,846_elevators_20_8.csv.gz,2.585094,2.838554,2.675399,3.545337,3.890592,3.106995,0.472032,0.998313,0.984571,...,0.422100,0.415784,0.005098,0.683593,0.705718,0.756705,0.650520,0.709583,0.701224,0.031754
